## Imports

In [25]:
#Imports
import ipdb
#%pdb

import pandas as pd
import cv2

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import sys

from sklearn.model_selection import train_test_split

import csv, random, numpy as np
from keras.models import load_model, Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.preprocessing.image import img_to_array, load_img, flip_axis, random_shift

%matplotlib inline

## Constants

In [26]:
#Constants

#Paths
PATH_FOLDER = 'Training_Data/Udacity_Training_Data/'
PATH_CSV = 'driving_log.csv'

#Image 
IMAGE_CUT_TOP_HEIGHT = 55
IMAGE_CUT_DOWN_HEIGHT = 25
IMAGE_RESIZE_WIDTH = 100
IMAGE_RESIZE_HEIGHT = 100

#Camera
CAMERA_LEFT_RIGHT_OFFSET = 0.2

#Chances for Augmentation
CHANCES_DARKEN = 0
CHANCES_SHIFT = 0
CHANCES_FLIP = 0

#Further Parameters
SPEED_MINIMUM = 20

## Read CSVs

In [27]:
#Read CSV
def read_csv(path):
    X, y = [], [] 
    
    csv = pd.read_csv(path)
    
    #Throw away slow instances
    csv = csv[(csv['speed']>SPEED_MINIMUM)]

    for index, row in csv.iterrows():
        #center
        X.append(row['center'].strip())
        y.append(row['steering'])
        #left
        X.append(row['left'].strip())
        y.append(row['steering']+CAMERA_LEFT_RIGHT_OFFSET)
        #right
        X.append(row['right'].strip())
        y.append(row['steering']-CAMERA_LEFT_RIGHT_OFFSET)
        
    return X,y

## Read Images

In [28]:
#Read Images

def resize_and_normalize(img):
    #printing out some stats and plotting
    #print('This image is:', type(img), 'with dimesions:', img.shape)
    #print(img)
    #Cut Top and Bottom (sky and car)
    #img_cut = img[IMAGE_CUT_TOP_HEIGHT:img.shape[0]-IMAGE_CUT_DOWN_HEIGHT, :, :]
    
    img_cut = img[IMAGE_CUT_TOP_HEIGHT:160-IMAGE_CUT_DOWN_HEIGHT, :, :]

    #Resize to smaller image size
    img_resize = cv2.resize(img_cut, (IMAGE_RESIZE_WIDTH, IMAGE_RESIZE_HEIGHT), interpolation=cv2.INTER_AREA)
            
    #Normalizing to a range of -0.5 to +0.5
    img_norm = (img_resize / 255. - .5).astype(np.float32)

    return img_norm

#image = cv2.imread(PATH_FOLDER+'IMG/center_2016_12_01_13_30_48_287.jpg')
#plt.imshow(resize_image(image))

## Augmentation

In [29]:
def augmentation(path, steering, augment, shape=(100,100)):    

    #Load
    image = cv2.imread(path)
    
    #Darken
    if random.random() < CHANCES_DARKEN:
        random_darken(image)
    
    #Shift
    if random.random() < CHANCES_SHIFT:
        random_shift(image, 0, 0.2, 0, 1, 2)
    
    #Flip
    if random.random() < CHANCES_FLIP:
        flip_axis(image,1)
        steering = steering * -1    
            
    #Resize
    image = resize_and_normalize(image)

    
    return image, steering
    
def random_darken(image):
    w, h = image.size

    # Create a random Box
    x1, y1 = random.randint(0, w), random.randint(0, h)
    x2, y2 = random.randint(x1, w), random.randint(y1, h)

    # Loop through Box and darken every pixel
    for i in range(x1, x2):
        for j in range(y1, y2):
            new_value = tuple([int(x * 0.5) for x in image.getpixel((i, j))])
            image.putpixel((i, j), new_value)
    return image

## Model

In [30]:
#Model
def model(load, shape, checkpoint=None):
    """Return a model from file or to train on."""
    if load and checkpoint: return load_model(checkpoint)

    conv_layers, dense_layers = [32, 32, 64, 128], [1024, 512]
    
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, activation='elu', input_shape=shape))
    model.add(MaxPooling2D())
    for cl in conv_layers:
        model.add(Convolution2D(cl, 3, 3, activation='elu'))
        model.add(MaxPooling2D())
    model.add(Flatten())
    for dl in dense_layers:
        model.add(Dense(dl, activation='elu'))
        model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer="adam")
    return model

In [31]:
def _generator(batch_size, X, y):
    """Generate batches of training data forever."""

    while 1:
        batch_X, batch_y = [], []
        for i in range(batch_size):
            
            sample_index = random.randint(0, len(X) - 1)
            sa = y[sample_index]       
            
            image, sa = augmentation(PATH_FOLDER+X[sample_index], sa, augment=True)
            batch_X.append(image)
            batch_y.append(sa)
        yield np.array(batch_X), np.array(batch_y)

In [32]:
if __name__ == '__main__':
    
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    net = model(load=False, shape=(IMAGE_RESIZE_WIDTH, IMAGE_RESIZE_HEIGHT, 3))
    X,y = read_csv(PATH_FOLDER+PATH_CSV)
    net.fit_generator(_generator(256, X, y), samples_per_epoch=20224, nb_epoch=2)
    net.save('checkpoints/short.h5')

Epoch 1/2
20224/20224 [==============================] - 396s - loss: 0.0239     